In [16]:
import random
from names_dataset import NameDataset, NameWrapper
from sklearn.cluster import KMeans
import numpy as np
nd = NameDataset()

class Perfil:
    def __init__(self, nome, altura, peso, idade, hobbies):
       
        self.nome = nome
        self.altura = altura
        self.peso = peso
        self.idade = idade
        self.hobbies = hobbies
        self.likes = 0

    def dar_like(self):
        self.likes += 1

    def __str__(self):
        return f"Nome: {self.nome}, Altura: {self.altura}m, Peso: {self.peso}kg, Idade: {self.idade} anos, Hobbies: {', '.join(self.hobbies)}"


class TinderSimulador:
    def __init__(self, usuario):
        
        self.usuario = usuario
        self.perfis = self._gerar_perfis_ficticios()
        self.interagidos = set()
        self.kmeans = None  


    def _gerar_perfis_ficticios(self):
        n = 40

        nomes = nd.get_top_names(n=n, country_alpha2='BR')
        nomes = random.sample(nomes['BR']['F']+nomes['BR']['M'],n)
        hobbies_lista = ["ler", "correr", "nadar", "viajar", "cozinhar", "dançar", "caminhar", "jogar video games"]

        perfis = []
        for nome in nomes:
            altura = round(random.uniform(1.50, 1.90), 2)
            peso = random.randint(50, 90)
            idade = random.randint(18, 35)
            hobbies = random.sample(hobbies_lista, random.randint(1, 4))
            perfis.append(Perfil(nome, altura, peso, idade, hobbies))
        
        return perfis

    def mostrar_perfil(self):
        for perfil in self.perfis:
            if perfil not in self.interagidos:
                print(perfil)
                acao = input("Digite 'like' para curtir, 'dislike' para não curtir ou 'finalizar' para terminar: ").strip().lower()
                if acao == "like":
                    perfil.dar_like()
                    self.interagidos.add(perfil)
                elif acao == "dislike":
                    self.interagidos.add(perfil)
                elif acao == "finalizar":
                    self.finalizar_interacao()
                    return
                else:
                    print("Comando inválido!")
            if len(self.interagidos) == len(self.perfis):
                print("Todos os perfis já foram visualizados!")
                break

        
    def segmentar_perfis(self):
        # Criando matriz de dados para clustering
        data = []
        
        # Convertendo hobbies para representação binária
        hobbies_lista = ["ler", "correr", "nadar", "viajar", "cozinhar", "dançar", "caminhar", "jogar video games"]
        
        for perfil in self.perfis:
            hobbies_binarios = [1 if h in perfil.hobbies else 0 for h in hobbies_lista]
            data.append([perfil.altura, perfil.peso, perfil.idade] + hobbies_binarios)
        
        data = np.array(data)
        
        # Segmentando com K-means
        self.kmeans = KMeans(n_clusters=3)  # Defini 3 clusters como exemplo, mas você pode ajustar
        clusters = self.kmeans.fit_predict(data)
        # Calculando médias para cada cluster
        medias = []
        for cluster_id in set(clusters):
            cluster_data = data[clusters == cluster_id]
            medias.append(cluster_data.mean(axis=0))
        
        # Mostrando as médias para cada cluster
        for i, media in enumerate(medias):
            print(f"Crushes grupo: {i+1}:")
            print("Altura média:", media[0])
            print("Peso médio:", media[1])
            print("Idade média:", media[2])
            print("Frequência de hobbies:")
            for j, hobby in enumerate(hobbies_lista):
                print(f"{hobby}: {media[3+j]}")
            print("\n")    


    def analisar_likes(self):
        likes_usuario = [perfil for perfil in self.perfis if perfil.likes > 0]
        if not likes_usuario:
            print("Você não deu like em nenhum perfil ainda.")
            return
        
        if self.kmeans is None:
            print("Você precisa segmentar os perfis antes de analisar os likes.")
            return
        
        # Convertendo likes do usuário para matriz
        data = []
        hobbies_lista = ["ler", "correr", "nadar", "viajar", "cozinhar", "dançar", "caminhar", "jogar video games"]
        
        for perfil in likes_usuario:
            hobbies_binarios = [1 if h in perfil.hobbies else 0 for h in hobbies_lista]
            data.append([perfil.altura, perfil.peso, perfil.idade] + hobbies_binarios)
        
        data = np.array(data)
        
        # Encontrar cluster mais próximo dos perfis que o usuário deu like
        clusters = self.kmeans.predict(data)
        cluster_counts = np.bincount(clusters)
        cluster_mais_proximo = np.argmax(cluster_counts)
        
        print(f"Análise dos seus likes:")
        print(f"O grupo de crushes mais próximo dos perfis que você deu like é o grupo {cluster_mais_proximo + 1}")

    
    def finalizar_interacao(self):
        self.perfis.sort(key=lambda x: x.likes, reverse=True)
        print("Interação finalizada. Na próxima vez, os perfis serão mostrados com base nas interações anteriores.")
        self.segmentar_perfis()   
        self.analisar_likes() 

def main():
    usuario = input("Digite seu nome para começar: ")
    tinder = TinderSimulador(usuario)
    while True:
        acao = input("\nDigite 'mostrar' para ver um perfil ou 'finalizar' para terminar: ").strip().lower()

        if acao == "mostrar":
            tinder.mostrar_perfil()
        elif acao == "finalizar":
            tinder.finalizar_interacao()
            break
        else:
            print("Comando inválido!")
         

if __name__ == "__main__":
    main()


Digite seu nome para começar: I

Digite 'mostrar' para ver um perfil ou 'finalizar' para terminar: mostrar
Nome: Matheus, Altura: 1.61m, Peso: 89kg, Idade: 28 anos, Hobbies: dançar, caminhar, jogar video games, cozinhar
Digite 'like' para curtir, 'dislike' para não curtir ou 'finalizar' para terminar: dislike
Nome: Marcos, Altura: 1.78m, Peso: 78kg, Idade: 33 anos, Hobbies: nadar, ler, dançar, jogar video games
Digite 'like' para curtir, 'dislike' para não curtir ou 'finalizar' para terminar: dislike
Nome: Sandra, Altura: 1.52m, Peso: 50kg, Idade: 21 anos, Hobbies: jogar video games, dançar, nadar
Digite 'like' para curtir, 'dislike' para não curtir ou 'finalizar' para terminar: like
Nome: Antonio, Altura: 1.59m, Peso: 78kg, Idade: 26 anos, Hobbies: nadar
Digite 'like' para curtir, 'dislike' para não curtir ou 'finalizar' para terminar: dislike
Nome: Lucas, Altura: 1.65m, Peso: 71kg, Idade: 25 anos, Hobbies: viajar, correr, caminhar, dançar
Digite 'like' para curtir, 'dislike' para não